In [1]:
import numpy as np
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import time
from matplotlib.animation import FuncAnimation
from IPython import display
import random
import statistics as stat
%run Backend_Retrieval.ipynb

In [2]:
def low_stdv_find(lst, rang):
    track = []
    track2 = []
    for i in range(len(lst)-rang):
        track.append(stat.stdev(lst[i:i+rang]))
        track2.append(i)
    ident = track2[track.index(min(track))]
    return lst[ident:ident+rang]

In [3]:
####################################################################
#          PARAMS
####################################################################
start_time=time.perf_counter()

L= 700        
steps_setup= 100               
splim = 15


des_speed=splim    
des_speed_inv=1/splim     
del_t= 0.1                   
acc_exp= 4                  
time_gap= 1               
min_gap= 2                  
comf_decel= 1.5               
acc_max= 1                 
length = 2                                          
 

#Parameter set-up
params=[]
params.append(des_speed_inv)
params.append(acc_exp)
params.append(time_gap)
params.append(comf_decel)
params.append(min_gap)
params.append(acc_max)
params.append(del_t)
params.append(length)
params.append(L)
####################################################################
#          PARAMETER SUMMARY
####################################################################
#  SETUP:
# acc exponent  = 4
# time_gap      = 1
# min space gap = 2
# comf decel    = 1.5
# max accel     = 1
# car length    = 2
####################################################################

In [ ]:
####################################################################
#   BASIC SIMULATION COLLECTING GLOBAL METRICS AND ITTERATING PARAMS
####################################################################
#  SETUP:
# iterating 1 to 150 cars, normally well past jam point
# 1000 steps at 0.1 secs 
# Road length 625m(force jam) with all cars starting even distance in first 600m
# 15 m/s speed limit
####################################################################
#  PARAMS TO ITERATE OVER:
# acc exponent
# min distance(safety gap)
# comf decel
# max acel
####################################################################


# PLAN:

# RUN BASIC SIM AS ABOVE OVER 100 VALUES OF DESIRED PARAM(FIRST ACC EXPONENT)

# SAVE RESULTS IN 3x150x100 MATRIX OUTSIDE TRIPLE NESTED FOR LOOP PREPPED FOR PLOTTING

# PLOT 50 EVENLY SPACED VALUES ON FUNDAMENTAL DIAGRAMS

# INSIDE SIM:

# DETERMINE SET OF 100 VALS IN global_flow_list[N,:] WITH LOWEST STANDARD DEVIATION AND AVERAGE THIS
# USE THIS VALUE IN SAVE RESULTS MATRIX,,,,,,100 vals used after observing behaviour of flow at fixed density

# COMPARE FINAL DATA OF PARAM WITH COVARIANCE

####################################################################
Min_Dist_itter = np.linspace(0.5,6,10)
ind = -1
L = 700
MinDist_Variation = np.zeros((2,149,10))

for xx in Min_Dist_itter:
    ind += 1
    params[4] = xx
    start = 1
    stop = 150
    #flow_global_list = np.zeros(stop-1,)
    #dens_global = np.zeros(stop-1)
    #avg_vel_global = []
    for N in range(60,300):
        carflow = []
        caravvel = []
        vel= np.zeros(N)     
        pos = np.zeros(N)
        start = np.linspace(0,600,N)
        for j in range(N):
            pos[j]=start[j]       
        acc= np.zeros(N)      
        headway = np.zeros(N)
        dv = np.zeros(N)
        for i in range(N):
            DisGap = 0
            if i+1!=N:
                if pos[i]>(pos[i+1]-length):
                    DisGap = (pos[i+1]-length-pos[i]+L)
                elif pos[i]<(pos[i+1]-length):
                    DisGap = (pos[i+1]-length-pos[i])
                dv[i] = vel[i]-vel[i+1]
            else:
                if pos[i]>(pos[0]-length):
                    DisGap = (pos[0]-length-pos[i]+L)
                else:
                    DisGap = (pos[0]-length-pos[i])
                dv[i] = vel[i]-vel[0]
            headway[i] = DisGap
        posnew= np.zeros(N)        
        velnew= np.zeros(N)
        time_pass=0

        for i in range(4000):
            if i%500==0:
                print(f'step {i}')
            
            posnew, velnew, acc = upd_pos_vel(N, pos, vel, acc, headway, dv, posnew, velnew, params)

            pos, vel, headway, dv = update_cars(N, pos, vel, posnew, velnew, headway, dv, params)

            den,flo = flow_global(N,vel,L)
            #dens_global[N-1] = den
            carflow.append(flo)
            count = 0
            for n in range(N):
                count += vel[n]
            caravvel.append(count/N)    
            time_pass+=del_t
    
        eq_flow = low_stdv_find(carflow,100)
        MinDist_Variation[0,N-1,ind] = stat.mean(eq_flow)
        
        eq_vel = low_stdv_find(caravvel,100)
        MinDist_Variation[1,N-1,ind] = stat.mean(eq_vel)
           
        print(f'{N} completed')
    print(f'{ind} PARAM VARIO COMPILED')

np.save('Data Storage\Varying Min Distance Gap 5', MinDist_Variation)

step 0
step 500
step 1000
step 1500
step 2000
step 2500
step 3000
step 3500
60 completed
step 0
step 500
step 1000
step 1500
step 2000
